The purpose of this notebook is to find the maximum of a function $J(\phi(x))$, where $\phi(x)$ is the output of a neural network, using automatic differentiation.
Here $J(a, b)=a-a^2+b-b^2+ab$ and $\phi(x_1, x_2)=(x_1-x_2, x_1+x_2)$, thus $J(\phi(x))$ has only one maximum reached for $x=(1, 0)$

In [2]:
import numpy as np
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense

2025-05-16 09:26:52.036966: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-16 09:26:52.080309: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

2025-05-16 09:26:53.313895: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
N = 10000
X = np.random.random((N, 2))
Y = np.zeros_like(X)
Y[:, 0] = X[:, 0] - X[:, 1]
Y[:, 1] = X[:, 0] + X[:, 1]

In [4]:
model = Sequential([Dense(2, input_shape=(2,))])
model.compile(optimizer='adam', loss='mse')

/home/pits/multi_agents_system_SIT/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-05-16 09:26:57.300837: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-05-16 09:26:57.300868: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:171] verbose logging is disabled. Rerun with verbose logging (usually --v=1 or --vmodule=cuda_diagnostics=1) to get more diagnostic output from this module
2025-05-16 09:26:57.300873: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:176] retrieving CUDA diagnostic information for host: 53a70ea1e15d
2025-05-

In [5]:
model.fit(X, Y, epochs=50, verbose=False)

In [6]:
def get_grad(x1, x2):
    with tf.GradientTape() as tape:
        a, b = model(tf.reshape([x1, x2], [1, 2]))[0]
        z = a - a*a + b - b*b + a*b
    return tape.gradient(z, {"x1": x1, "x2": x2})


In [8]:
x1 = tf.Variable(10.0)
x2 = tf.Variable(20.0)
learning_rate = 0.01
for _ in range(1000):
    grad = get_grad(x1, x2)
    x1.assign_add(learning_rate*grad["x1"])
    x2.assign_add(learning_rate*grad["x2"])
print(x1, x2)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.0000028610229492> <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=5.657527069047319e-08>
